# Introduction

This notebook will focus primarily statistical side of the time series modelling, including how to define Correlation's, the concept of Stationarity, Auto-Correlation & Partial Auto-Correlation Function, Auto-Regressive & Moving Average Processes in modelling and how to perform Model Diagnostics.

<b>Interesting Read : </b>[STAT 510](https://online.stat.psu.edu/stat510/)

<img src='../Materials/stat_501.png' width='550' align='left'>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>

# Imports

In [ ]:
# General
from warnings import filterwarnings
filterwarnings('ignore')
from cycler import cycler


# Data Wrangling
import pandas as pd
import numpy as np

# Data Viualization
import matplotlib.pyplot as plt
import seaborn as sns
import mplfinance as mpl

# Time Series Specific
from statsmodels.tsa.seasonal import seasonal_decompose


# Datetime
from datetime import datetime

# DataHandler
from helperhandler import dataHolder

# Path and Variable Initialisation

In [ ]:
root_path = '../'
raw_datapath = root_path+'Raw Data/'
prepared_datapath = root_path+'Prepared Data/'

In [ ]:
plt.rcParams['figure.dpi'] = 100
plt.rcParams['figure.figsize'] = (15,7)
plt.style.use('ggplot')

# Load & Explore the data

In [ ]:
dataHolder.load_data()

In [ ]:
dataHolder.dataDf

## USA Consumer, Income, Production, Savings & Unemployment Data

In [ ]:
k='usa_economic'
print(dataHolder.bucket[k].long_description)
dataHolder.bucket[k].data.head()

In [ ]:
dataHolder.bucket[k].exploratory_plot()

## Air Passengers Data

In [ ]:
k='airp_data'
print(dataHolder.bucket[k].long_description)
dataHolder.bucket[k].data.head()

In [ ]:
dataHolder.bucket[k].exploratory_plot()

## Beer Production Data

In [ ]:
k='beer_prod'
print(dataHolder.bucket[k].long_description)
dataHolder.bucket[k].data.head()

In [ ]:
dataHolder.bucket[k].exploratory_plot()

## Britannia Stock Data

In [ ]:
k='brit_stock'
print(dataHolder.bucket[k].long_description)
dataHolder.bucket[k].data.head()

In [ ]:
dataHolder.bucket[k].exploratory_plot()

# Correlation

**Correlation** : Two variables are said to be correlated when the value assumed by one affects the distribution of the other. It reflects the association between the two variables whose strength usually lies within the range of -1 to +1. If, as the value of X increase there is an increase in Y, then X & Y are said to be positively correlated. Also if, as the value of X decrease there is an increase in Y, then X & Y are said to be negatively correlated.

Different Types Of Correlations

<img src='https://miro.medium.com/max/2000/1*cxBhYwEBPLvs0E8E7Ehe_g.png' width='500'>

- Pearson Correlation : Quantifying association between Two continuous features.
\begin{equation}
    \rho_{p}=\frac{Cov(X,Y)}{\sigma_{x} \sigma_{y}} = \frac{\sum_{i=1}^{n}((x_{i}-\bar{x})(y_{i}-\bar{y}))}{\sqrt{\sum_{i=1}^{n}(x_{i}-\bar{x})^2 }*{\sqrt{\sum_{i=1}^{n}(y_{i}-\bar{y})^2 }}}
\end{equation}


- Spearman Correlation : Quantifying `RANK` association between ordinal & continuous features.
\begin{equation}
    \rho_{s}=\frac{Cov(rank(X),rank(Y))}{\sigma_{rank(x)} \sigma_{rank(y)}} = \frac{\sum_{i=1}^{n}((rank(x_{i})-rank(\bar{x}))(rank(y_{i})-rank(\bar{y}))}{\sqrt{\sum_{i=1}^{n}(rank(x_{i})-rank(\bar{x}))^2 }*{\sqrt{\sum_{i=1}^{n}(rank(y_{i})-rank(\bar{y}))^2 }}}
\end{equation}


- Kendall Tau Correlation : Quantifying `RANK` association between ordinal & continuous features, works quite well with Non-Normally distributed data.

\begin{equation}
    \tau_{k} = \frac{\sum_{i=1}^{n} \sum_{j=1}^{n} signum(x_{i}-x_{j}) signum(y_{i}-y_{j})}{n(n-1)}
\end{equation}

- Point Biserial Correlation : Quantifying association between Nominal and Continous Feature.

\begin{equation}
    r_{pb} = \frac{M_{1}-M_{0}}{s_{n}}\sqrt{\frac{n_{0}n_{1}}{n_{2}}}
\end{equation}


    - Here > M1 : Mean of target across 1 category, ; M2 : Mean of the target across 0 category; sn : Standard Deviation in the data; n0, n1: count of the number of datapoints in each category; n : Total number of datapoints.

- Annova eta values : Quantifying association between Two nominal features.

[A comparision of the Correlations](https://ag-ds-bubble.medium.com/correlations-pearson-correlation-is-not-one-solution-for-all-25f7220220fe)


****

But be wary of the correlations though, because `Correlation` doesnt always translation into `Causation`

[Spurious Correlations](https://tylervigen.com/spurious-correlations)
<img src='../Materials/Spurious Correlation.png'>



In [ ]:
from scipy.stats import pearsonr, kendalltau, spearmanr, pointbiserialr

In [ ]:
np.random.seed(20)
n = 10
x=np.arange(n)
y=x+np.random.uniform(low=-1,high=1, size=len(x))*2

# x = x/np.linalg.norm(x)
# y = y/np.linalg.norm(y)

_=plt.scatter(x,y)
_=plt.plot(x,x,c='r')

In [ ]:
x_mean = np.mean(x)
y_mean = np.mean(y)

x_deviat = x-x_mean
y_deviat = y-y_mean

numerator = sum(x_deviat*y_deviat)/(n)
denominator = np.sqrt(sum(x_deviat**2)/n)*np.sqrt(sum(y_deviat**2)/n)
pearson_corr = numerator/denominator

pearson_corr, pearsonr(x,y)

In [ ]:
np.corrcoef(x,y)

In [ ]:
pd.DataFrame(np.c_[x,y]).corr()

In [ ]:
usa_cipsu_data = dataHolder.bucket['usa_economic'].data.copy()

In [ ]:
usa_cipsu_data.corr(method='pearson')

In [ ]:
pearsonr(usa_cipsu_data.Production, usa_cipsu_data.Unemployment)

In [ ]:
kendalltau(usa_cipsu_data.Production, usa_cipsu_data.Unemployment)

In [ ]:
spearmanr(usa_cipsu_data.Production, usa_cipsu_data.Unemployment)

In [ ]:
# Generating Correlation Heatmaps
corr_df = usa_cipsu_data.corr(method='pearson')
corr_df = corr_df.style
corr_df = corr_df.background_gradient(cmap=sns.light_palette("red", as_cmap=True))
corr_df = corr_df.highlight_max(color='black')
corr_df

In [ ]:
lagged_data = pd.DataFrame()
for elag in range(5):
    for ecol in ['Consumption', 'Income']:
        lagged_data[ecol+'_{0}'.format(elag)] = usa_cipsu_data['Consumption'].shift(elag)
lagged_datacorr=lagged_data.corr()
lagged_datacorr = lagged_datacorr[['Income_0', 'Income_1', 'Income_2', 'Income_3', 'Income_4']]
lagged_datacorr = lagged_datacorr[lagged_datacorr.index.isin(['Consumption_0','Consumption_1','Consumption_2',
                                              'Consumption_3','Consumption_4'])]
lagged_datacorr

In [ ]:
lagged_datacorrst = lagged_datacorr.copy().style
lagged_datacorrst = lagged_datacorrst.background_gradient(cmap=sns.light_palette("red", as_cmap=True))
lagged_datacorrst = lagged_datacorrst.highlight_max(color='black')
lagged_datacorrst

In [ ]:
_=sns.heatmap(lagged_datacorr, annot=True)

# Autocorrelation

Since Time Series, of any Object - being measured, is something which is not `expected` to abruptly shift from its trajectory/course which stems a notion of previous `timestamped` values having some association/correlation with the values in very near future/ in a particular seasonal patter to arise. This concpet is something we can take advantage of for predicting the future.

Use of AutoCorrelation can be Multiple :- 

- Figuring out which model to use
- Estimating the parameters of the models
- Understanding the Underlying structure of the Time Series

****
There are two forms of Auto-Correlation that are used in Time Series Analysi

- ***Auto-Correlation*** : Auto-Correlation as designed above is something which lets us gauge into the internal structure as to how are the previous datapoints related to the ones that are to come. But the catch here is that, 

Suppose we are calcluate Auto - Correlation between 
  
1)  AC between $y_{t}$ and $y_{t-1}$  = 0.78

2)  AC between $y_{t}$ and $y_{t-2}$ = 0.76

And we have the following conversation :- 


**Me :)** Were we to use both of these variables as predictors, since $y_{t-1}$ is already explaining 78% of variance(correlation, for simplicity) of the $y_{t}$, would it be even helpful to use $y_{t-2}$?

**You:)** Well it is having 0.76 correlation, maybe? 🧐

**Me:)** Ok, then i will reverse my question, if $y_{t-2}$ is having 0.76 correlation, would it be wise for us to even consider the $y_{t-1}$ series?

**You:)** Well I dont really know, I understand that the there will be effect of $y_{t-2}$ in the $y_{t-1}$ series but how do i quantify that?

**Me:)** PARTIAL AUTO CORRELATION 

**You:)** 🤩🤩🤩🤩, wait, what is Partial Auto Correlation and how do i quantify it?


Getting a value of correlation neglecting the compunding effect of the series..

- ***Partial Auto-Correlation*** : 

**Me:)** Effectively what we need from Partial Auto-Correlation is for us to tell that what is the effect of `Individual Lagged Time Series` $y_{t-n}$ on the current time series $y_{t}$. So this is what we do :-

- 1.) Take the series $y_{t-1}$ and  $y_{t}$
- 2.) Regress $y_{t-1}$ on  $y_{t}$ and note the Correlation between the two.
- 3.) Take the residuals(Whatever $y_{t-1}$ wasnt able to explain of $y_{t}$) of that regression, basically the errors
- 4.) Now regress the $y_{t-2}$ on the `errors`, and repeat...

[Partial Auto Correlation](https://towardsdatascience.com/understanding-partial-auto-correlation-fa39271146ac)

Lets jump to code!


<b>Auto-Correlation</b>

In [ ]:
from statsmodels.graphics.tsaplots import acf, plot_acf, pacf, plot_pacf

In [ ]:
tempdata = lagged_data[[k for k in lagged_data.columns if 'Con' in k]]
tempdata.corr().iloc[:,0]

In [ ]:
_=plot_acf(lagged_data.Consumption_0)
acf(lagged_data.Consumption_0)[:5]

<b>Partial Auto-Correlation</b>

In [ ]:
from sklearn import linear_model

def calc_pacf(data, lags = 2):
    data.name='y'
    data = data.to_frame()
    pcorr=[]
    
    if lags==1:
        return [1]
    elif lags==2:
        data['y_L1'] = data.y.shift(1)
        return [1, data.corr().values[1][0]]
    else:
        data['y_L1'] = data.y.shift(1)
        pcorr = [1, data.corr().values[1][0]]
        
        for _l in range(2,lags):
            _tlag_col = 'y_L_{0}'.format(_l)
            calcData = data.y.to_frame().copy()
            calcData[_tlag_col] = calcData.y.shift(_l)
            reg_cols = []
            for _ll in range(1,_l):
                scol = 'y_L_{0}'.format(_ll)
                calcData[scol] = calcData.y.shift(_ll)
                reg_cols.append(scol)
            calcData = calcData.dropna()
            
            # Fit the first Linear Model on t_i
            _model = linear_model.LinearRegression()
            _model.fit(calcData[reg_cols], calcData['y'])
            _resid1 = calcData['y']-_model.predict(calcData[reg_cols])
            # Fit the first Linear Model on t_l
            _model = linear_model.LinearRegression()
            _model.fit(calcData[reg_cols], calcData[_tlag_col])
            _resid2 = calcData[_tlag_col]-_model.predict(calcData[reg_cols])
            pcorr.append(pearsonr(_resid1.values, _resid2.values)[0])
            
        return pcorr


In [ ]:
calc_pacf(lagged_data.Consumption_0,10)

In [ ]:
_=plot_pacf(lagged_data.Consumption_0, lags=10)
pacf(lagged_data.Consumption_0)[:10]

# Stationarity

Dealing with Time Series Models come with some assumptions, especially univariate models, in which `Stationarity` is one of the primary requirements.

If a Time Series is Stationary, 
- The Time Series $y_{t}$ has a constant mean, i.e $E[Y_{t}]=const$
- The Time Series $y_{t}$ has a constant variance, i.e $Var[Y_{t}]=const$
- The Time Series $y_{t}$ doesnt exhibit seasonality.

How to detect if a Time Seiries is Stationary?
- Visual Inspection of the Time Series Plot
- Local & Global Tests
- Unit Root Tests - `ADF` Test & `KPSS` Test

In [ ]:
airpassengers_data = dataHolder.bucket['airp_data'].data.copy()
beerprod_data = dataHolder.bucket['beer_prod'].data.copy()
britanniastock_data = dataHolder.bucket['brit_stock'].data.copy()

In [ ]:
def get_splot1():
    fig, axes = plt.subplots(2,3, figsize=(20,10))
    _=usa_cipsu_data.Consumption.plot(ax=axes[0,0])
    _=axes[0,0].set_xlabel('')
    _=axes[0,0].set_ylabel('US Consumption pct change')
    _=usa_cipsu_data.Income.plot(ax=axes[0,1])
    _=axes[0,1].set_xlabel('')
    _=axes[0,1].set_ylabel('US Income pct change')
    _=usa_cipsu_data.Unemployment.plot(ax=axes[0,2])
    _=axes[0,2].set_xlabel('')
    _=axes[0,2].set_ylabel('US Unemployment pct change')


    _=airpassengers_data.Passengers.plot(ax=axes[1,0])
    _=axes[1,0].set_xlabel('')
    _=axes[1,0].set_ylabel('Passengers')
    _=beerprod_data.MBP.plot(ax=axes[1,1])
    _=axes[1,1].set_xlabel('')
    _=axes[1,1].set_ylabel('Beer Production')
    _=britanniastock_data.Volume.plot(ax=axes[1,2])
    _=axes[1,2].set_xlabel('')
    _=axes[1,2].set_ylabel('Britannia Stock Volume')

In [ ]:
get_splot1()

Of these Time Series which do you think is Stationary?

In [ ]:
# Unit Root Tests
from statsmodels.tsa.stattools import adfuller, kpss

def adf_test(timeseries):
    dftest = adfuller(timeseries, autolag='AIC')
    dfoutput = pd.Series(dftest[0:4], index=['Test Statistic','p-value','#Lags Used','Number of Observations Used'])
    for key,value in dftest[4].items():
           dfoutput['Critical Value (%s)'%key] = value
    return dfoutput.to_frame()

def kpss_test(timeseries):
    kpsstest = kpss(timeseries, regression='c', nlags="auto")
    kpss_output = pd.Series(kpsstest[0:3], index=['Test Statistic','p-value','Lags Used'])
    for key,value in kpsstest[3].items():
        kpss_output['Critical Value (%s)'%key] = value
    return kpss_output.to_frame()

In [ ]:
adf_test(usa_cipsu_data.Consumption)

In [ ]:
kpss_test(usa_cipsu_data.Consumption)

# Statistical Tests

- Test for Auto Correlation
- Test for Seasonality (Strength)
- Test for Trend (Strength)
- Test for Stationarity
- Test for Distribution

In [ ]:
import statsmodels.api as sm

from statsmodels.stats.diagnostic import acorr_ljungbox
from statsmodels.stats.stattools import durbin_watson, jarque_bera
from statsmodels.tsa.stattools import adfuller, kpss
from statsmodels.graphics.gofplots import qqplot
from statsmodels.tsa.seasonal import seasonal_decompose

from scipy import stats

## Quantifying the Seasonality & Trend
[Strength of Seasonality & Trend](https://otexts.com/fpp2/seasonal-strength.html#seasonal-strength)

In [ ]:

def get_decompose_plot(data):
    data = data.copy()
    data.index.name = ''
    grid = plt.GridSpec(4, 2, wspace=0.1, hspace=0.5)
    series_ax = plt.subplot(grid[0:2, :])
    series_ax.set_title('Time Series')
    
#     ts_trnsfrm, blambda = boxcox(data['ts'])
#     data['ts'] = pd.Series(ts_trnsfrm, name='ts', index=data['ts'].index)
    data['ts'].plot(ax=series_ax)

    trend_ax = plt.subplot(grid[2, 0])
    trend_ax.set_title('Trend Series')
    data['Trend'].plot(ax=trend_ax)

    cycl_ax = plt.subplot(grid[2, 1])
    cycl_ax.set_title('Cyclic Component')
    data['Cyclicity'].plot(ax=cycl_ax)
    
    seas_ax = plt.subplot(grid[3, 0])
    seas_ax.set_title('Seasonal Component')
    data['Seasonality'].plot(ax=seas_ax)

    resid_ax = plt.subplot(grid[3, 1])
    resid_ax.set_title('Residuals/Noise')
    data['Residual'].plot(ax=resid_ax)
        
    return grid


In [ ]:
tsdata = airpassengers_data.Passengers
decomp=seasonal_decompose(tsdata, period=12, extrapolate_trend=True, model='multiplicative')

decomData = pd.DataFrame(columns=['Trend',  'Cyclicity', 'Seasonality', 'Residual'],
                         index=tsdata.index)

tt = decomp.trend
st = decomp.seasonal
rt = decomp.resid

decomData['Trend'] = tt
decomData['Seasonality'] = st
decomData['Residual'] = rt
decomData['ts'] = tsdata

get_decompose_plot(decomData)

In [ ]:
_tmeasure=np.var(rt)/np.var(tt+rt)
_tmeasure = max([0, 1-_tmeasure])
print('Strength of Trend is : ', round(_tmeasure,4))

In [ ]:
_measure=np.var(rt)/np.var(st+rt)
_measure = max([0, 1-_measure])
print('Strength of Seasonal is : ', round(_measure,4))

## Tests for Auto-Correlation
- Durbin Watson Test
     - **Null Hypothesis :** There is no serial correlation in the residuals
     - **Alternate Hypothesis :** Residuals follow an AR1 process
     - **In Simple Terms :** If `p-value`<0.05(or any other significance value) then the auto-correlation is present, else if `p-value`>0.05 auto-correlation is absent.
     
     
     
- Ljung-Box Test (More Accurate)
     - **Null Hypothesis :** The residuals are independently distributed.
     - **Alternate Hypothesis :** The residuals are not independently distributed; they exhibit serial correlation.
     - **In Simple Terms :** This tests statistics always results in values ranging in from 0 to 4, if the `test-statistics`=2, then this means that there is no serial autocorrelation, if the `test-statistics` is closer to 0 then it mean there can be `Positive Serial Correlation` or if it is towards 4 it can mean that there might be `Negative Serial Correlation`
     


In [ ]:
durbin_watson(britanniastock_data.Close)

In [ ]:
durbin_watson(usa_cipsu_data.Consumption)

In [ ]:
durbin_watson(beerprod_data.MBP)

<b>Ljung-Box Test</b>

In [ ]:
data = sm.datasets.sunspots.load_pandas().data
res = sm.tsa.ARMA(beerprod_data.MBP, (1,1)).fit(disp=-1)
acorr_ljungbox(res.resid, lags=[12], return_df=True)

In [ ]:
data = sm.datasets.sunspots.load_pandas().data
res = sm.tsa.ARMA(usa_cipsu_data.Consumption, (1,1)).fit(disp=-1)
acorr_ljungbox(res.resid, lags=[12], return_df=True)

## Tests For stationarity
- [Augmented Dickey Fuller (ADF) Test](https://www.youtube.com/watch?v=1opjnegd_hA)
     - **Null Hypothesis :** A unit root is present in a time series sample
     - **Alternate Hypothesis :** Depending on which version of the test is used, but is usually stationarity or trend-stationarity.
     - **In Simple Terms :** If ADF Test outputs `p-value`>0.05(or Significance level set by you), then the time series is non-stationary
     
     
- Kwiatkowski–Phillips–Schmidt–Shin (KPSS) Test
     - **Null Hypothesis :** The data is stationary.
     - **Alternate Hypothesis :** The data is non-stationary.
     - **In Simple Terms :** If KPSS Test outputs `p-value`<0.05(or Significance level set by you), then the time series is non-stationary.

In [ ]:
adf_test(beerprod_data.MBP)

In [ ]:
kpss_test(beerprod_data.MBP)

## Tests for Distributions

- Jarque Bera Test
     - **Null Hypothesis :** The data is following a Normal Distribution
     - **Alternate Hypothesis :** The data is following some other distribution.
     - **In Simple Terms :** If Jarque Bera Test outputs `p-value`<0.05(or Significance level set by you), then the time series is not following a Normal Distribution.

In [ ]:
def jb_test(timeseries):
    dftest = jarque_bera(timeseries)
    dfoutput = pd.Series(dftest, index=['JB', 'JBpv', 'skew', 'kurtosis'])
    dfoutput = dfoutput.to_frame()
    dfoutput.index.name = 'Measure'
    dfoutput.columns=['Value']
    return dfoutput

jb_test(usa_cipsu_data.Consumption)


In [ ]:
white_noise = np.random.normal(loc=3,size=int(1e5))
sns.distplot(white_noise)

In [ ]:
jb_test(white_noise)

<b>Q-Q Plot</b>

If the line is straight then the data is following the normal distribution

In [ ]:
_=qqplot(usa_cipsu_data.Consumption)

# Model Metrics

- Mean Absolute Percentage Error (MAPE):
\begin{equation}
    MAPE = \frac{1}{n} \sum_{i=1}^{n}\left | \frac{y_{t}-\hat y_{t}}{y_{t}} * 100\right|
\end{equation}
- Mean Squared Error (MSE):
\begin{equation}
    MSE = \frac{1}{n}\sum_{i=1}^{n}(y_{t}-\hat y_{t})^2
\end{equation}


> *Both MAPE & MSE dont have the sense of direction of the error, just the magnitude*

> *Both MAPE & MSE have their magnitude which is a reflectance of the error relative to actual target value*

> *MSE Penalises greater error more than the smaller error*

> *MAPE will become infinite if the actual data is 0*


- Custom Metrics : Based on the KPI's being tracked

<b>Effect on MSE as the prediction moves farther away from the actual</b>

In [ ]:
np.random.seed(4)
x=[1,2,3,4,10,12]
for ex in x:
    exp = ex+np.random.normal(scale=0.1,size=100)
    plt.scatter(exp, (exp-ex)**2, s=2)


<b>Effect on MAPE as the prediction moves farther away from the actual</b>

In [ ]:
np.random.seed(4)
x=[1,2,3,4,10,12]
for ex in x:
    exp = ex+np.random.normal(scale=0.1,size=100)
    plt.scatter(exp, 100*(abs(exp-ex)/ex),s=2)


<b>Calculation of the Metrics<b>

In [ ]:
from statsmodels.tsa.arima_model import ARIMA
ydata = dataHolder.bucket['beer_prod'].data.copy()
model=ARIMA(ydata.MBP, (1,0,1))
fit=model.fit()
fit.summary()

In [ ]:
ydata['Predicted'] = fit.predict(0,475)
ydata.plot()


In [ ]:
ydata['Residuals'] = ydata.MBP - ydata.Predicted
_=ydata.Residuals.plot()

In [ ]:
resid = ydata.Predicted-ydata.MBP
resid_scaled = resid/ydata.Predicted
resid_scaledabs = sum(abs(resid_scaled))
mape = 100*(resid_scaledabs/ydata.shape[0])
mape = round(mape, 3)
print('MAPE : ', mape, '%')

In [ ]:
resid = ydata.Predicted-ydata.MBP
resid_scaled = resid**2
resid_scaledabs = sum(abs(resid_scaled))
mse = 100*(resid_scaledabs/ydata.shape[0])
mse = round(mse, 3)
print('MSE : ', mse)

# Random Testing Space